<a href="https://colab.research.google.com/github/rhardaway/bigquery-e2e/blob/master/Query_CQL_with_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using LLMs to Generate CQL

Since LLMs seem to excel at a lot of things, we wanted to show how they can be used to generate CQL to query your Cassandra tables. This notebook provides a guide derived from the [SQL-PaLM](https://arxiv.org/abs/2306.00739) paper on how to automatically show the LLM your DB schema, and let it inform the LLM on querying your data.

## Setup

#### Requirements

In [ ]:
# Install requirements, if not already installed
!pip install openai cassandra-driver

#### Connect to Services

In [ ]:
# Initialize the OpenAI Client
import os

from getpass import getpass
import openai

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

client = openai.OpenAI()


OpenAI API Key: ··········


In [ ]:
# Connect to a Cassandra Cluster and initialize the session
import re

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from getpass import getpass
from google.colab import files

ASTRA_TOKEN = os.environ.get(
    "ASTRA_DB_TOKEN",
    getpass("Astra DB Token: ")
)

ASTRA_BUNDLE_PATH = os.environ.get(
    "ASTRA_DB_BUNDLE_PATH",
    list(files.upload().keys())[0],
)

ASTRA_KEYSPACE = os.environ.get(
    "ASTRA_DB_KEYSPACE",
    input("Astra DB Keyspace: "),
)

cloud_config = {
    'secure_connect_bundle': ASTRA_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider("token", ASTRA_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect(keyspace=ASTRA_KEYSPACE)


def execute_statement(statement: str):
    # This is a simple wrapper around executing CQL statements in our
    # Cassandra cluster, and either raising an error or returning the results
    try:
        rows = session.execute(statement)
        return rows.all()
    except:
        print(f"Query Failed: {statement}")
        raise


#### (Optional) Dummy DB Setup

Feel free to skip this section if you are instead adapting the notebook to fit your existing Cassandra Database. Here, we will utilize the python `cassandra-driver` package to connect to a DB and create some fake tables. This schema is pulled from [this DataStax example](https://www.datastax.com/learn/data-modeling-by-example/digital-library-data-model) on creating a data model for a digital music library.

In [ ]:
# Create all necessary tables
create_tables_cql = """CREATE TABLE performers (
    name TEXT PRIMARY KEY,
    type TEXT,
    country TEXT,
    born INT,
    died INT,
    founded INT
);

CREATE TABLE albums_by_performer (
    performer TEXT,
    year INT,
    title TEXT,
    genre TEXT,
    PRIMARY KEY (performer, year, title)
) WITH CLUSTERING ORDER BY (year DESC, title ASC);

CREATE TABLE albums_by_title (
    title TEXT,
    year INT,
    performer TEXT,
    genre TEXT,
    PRIMARY KEY (title, year)
) WITH CLUSTERING ORDER BY (year DESC);

CREATE TABLE albums_by_genre (
    genre TEXT,
    year INT,
    performer TEXT,
    title TEXT,
    PRIMARY KEY (genre, year, performer, title)
) WITH CLUSTERING ORDER BY (year DESC);

CREATE TABLE tracks_by_title (
    title TEXT,
    album_title TEXT,
    album_year INT,
    number INT,
    length INT,
    genre TEXT,
    PRIMARY KEY (title, album_title, album_year, number)
) WITH CLUSTERING ORDER BY (album_title ASC, album_year DESC, number ASC);

CREATE TABLE tracks_by_album (
    album_title TEXT,
    album_year INT,
    number INT,
    title TEXT,
    length INT,
    genre TEXT STATIC,
    PRIMARY KEY (album_title, album_year, number)
) WITH CLUSTERING ORDER BY (album_year DESC, number ASC);

CREATE TABLE users (
    id UUID PRIMARY KEY,
    name TEXT
);

CREATE TABLE tracks_by_user (
    id UUID,
    month DATE,
    timestamp TIMESTAMP,
    album_title TEXT,
    album_year INT,
    number INT,
    title TEXT,
    length INT,
    PRIMARY KEY (id, timestamp)
) WITH CLUSTERING ORDER BY (timestamp DESC);"""

In [ ]:
# This parses the text above into executable strings by the driver
for statement in create_tables_cql.split(";"):
    if len(statement.strip()):
        execute_statement(statement.strip())

In [ ]:
# Now populate with some fake data
insert_fake_data_cql = """
-- Insert data into performers
INSERT INTO performers (name, type, country, born, died, founded) VALUES ('The Beatles', 'Band', 'UK', 1960, NULL, 1960);
INSERT INTO performers (name, type, country, born, died, founded) VALUES ('Adele', 'Solo', 'UK', 1988, NULL, NULL);
INSERT INTO performers (name, type, country, born, died, founded) VALUES ('Elton John', 'Solo', 'UK', 1947, NULL, NULL);
INSERT INTO performers (name, type, country, born, died, founded) VALUES ('Queen', 'Band', 'UK', 1970, NULL, 1970);
INSERT INTO performers (name, type, country, born, died, founded) VALUES ('Taylor Swift', 'Solo', 'USA', 1989, NULL, NULL);

-- Insert data into albums by performer, title, and genre
-- Assuming 'Pop' as a genre for all for simplicity
INSERT INTO albums_by_performer (performer, year, title, genre) VALUES ('The Beatles', 1967, 'Sgt. Pepper''s Lonely Hearts Club Band', 'Pop');
INSERT INTO albums_by_performer (performer, year, title, genre) VALUES ('Adele', 2015, '25', 'Pop');
INSERT INTO albums_by_performer (performer, year, title, genre) VALUES ('Elton John', 1973, 'Goodbye Yellow Brick Road', 'Pop');
INSERT INTO albums_by_performer (performer, year, title, genre) VALUES ('Queen', 1975, 'A Night at the Opera', 'Pop');
INSERT INTO albums_by_performer (performer, year, title, genre) VALUES ('Taylor Swift', 2014, '1989', 'Pop');

-- Repeat for albums_by_title
INSERT INTO albums_by_title (title, year, performer, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 'The Beatles', 'Pop');
INSERT INTO albums_by_title (title, year, performer, genre) VALUES ('25', 2015, 'Adele', 'Pop');
INSERT INTO albums_by_title (title, year, performer, genre) VALUES ('Goodbye Yellow Brick Road', 1973, 'Elton John', 'Pop');
INSERT INTO albums_by_title (title, year, performer, genre) VALUES ('A Night at the Opera', 1975, 'Queen', 'Pop');
INSERT INTO albums_by_title (title, year, performer, genre) VALUES ('1989', 2014, 'Taylor Swift', 'Pop');

-- Repeat for albums_by_genre
INSERT INTO albums_by_genre (genre, year, performer, title) VALUES ('Pop', 1967, 'The Beatles', 'Sgt. Pepper''s Lonely Hearts Club Band');
INSERT INTO albums_by_genre (genre, year, performer, title) VALUES ('Pop', 2015, 'Adele', '25');
INSERT INTO albums_by_genre (genre, year, performer, title) VALUES ('Pop', 1973, 'Elton John', 'Goodbye Yellow Brick Road');
INSERT INTO albums_by_genre (genre, year, performer, title) VALUES ('Pop', 1975, 'Queen', 'A Night at the Opera');
INSERT INTO albums_by_genre (genre, year, performer, title) VALUES ('Pop', 2014, 'Taylor Swift', '1989');

-- Insert data into tracks_by_title and tracks_by_album
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Lucy in the Sky with Diamonds', 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 1, 208, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('With a Little Help from My Friends', 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 2, 163, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 3, 122, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Getting Better', 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 4, 174, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Fixing a Hole', 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 5, 139, 'Pop');

INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Hello', '25', 2015, 1, 295, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Send My Love', '25', 2015, 2, 223, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('I Miss You', '25', 2015, 3, 350, 'Pop');

INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Candle in the Wind', 'Goodbye Yellow Brick Road', 1973, 1, 219, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Bennie and the Jets', 'Goodbye Yellow Brick Road', 1973, 2, 323, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Goodbye Yellow Brick Road', 'Goodbye Yellow Brick Road', 1973, 3, 193, 'Pop');

INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Bohemian Rhapsody', 'A Night at the Opera', 1975, 1, 354, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Love of My Life', 'A Night at the Opera', 1975, 2, 220, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Youre My Best Friend', 'A Night at the Opera', 1975, 3, 178, 'Pop');

INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Welcome to New York', '1989', 2014, 1, 212, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Blank Space', '1989', 2014, 2, 231, 'Pop');
INSERT INTO tracks_by_title (title, album_title, album_year, number, length, genre) VALUES ('Style', '1989', 2014, 3, 230, 'Pop');

-- Repeat for tracks_by_album with corresponding track numbers
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 1, 'Lucy in the Sky with Diamonds', 208, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 2, 'With a Little Help from My Friends', 163, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 3, 'Sgt. Pepper''s Lonely Hearts Club Band', 122, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 4, 'Getting Better', 174, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Sgt. Pepper''s Lonely Hearts Club Band', 1967, 5, 'Fixing a Hole', 139, 'Pop');

INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('25', 2015, 1, 'Hello', 295, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('25', 2015, 2, 'Send My Love', 223, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('25', 2015, 3, 'I Miss You', 350, 'Pop');

INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Goodbye Yellow Brick Road', 1973, 1, 'Candle in the Wind', 219, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Goodbye Yellow Brick Road', 1973, 2, 'Bennie and the Jets', 323, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('Goodbye Yellow Brick Road', 1973, 3, 'Goodbye Yellow Brick Road', 193, 'Pop');

INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('A Night at the Opera', 1975, 1, 'Bohemian Rhapsody', 354, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('A Night at the Opera', 1975, 2, 'Love of My Life', 220, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('A Night at the Opera', 1975, 3, 'Youre My Best Friend', 178, 'Pop');

INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('1989', 2014, 1, 'Welcome to New York', 212, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('1989', 2014, 2, 'Blank Space', 231, 'Pop');
INSERT INTO tracks_by_album (album_title, album_year, number, title, length, genre) VALUES ('1989', 2014, 3, 'Style', 230, 'Pop');

-- Insert data into users
INSERT INTO users (id, name) VALUES (uuid(), 'John Doe');
INSERT INTO users (id, name) VALUES (uuid(), 'Jane Smith');
INSERT INTO users (id, name) VALUES (uuid(), 'Emily Johnson');
INSERT INTO users (id, name) VALUES (uuid(), 'Michael Brown');
INSERT INTO users (id, name) VALUES (uuid(), 'Jessica Davis');

-- Insert data into tracks_by_user
-- User ids should be copied from the users insert statements once generated
-- The following are placeholders and should be replaced with actual UUIDs
INSERT INTO tracks_by_user (id, month, timestamp, album_title, album_year, number, title, length) VALUES (uuid(), '2024-01-01', toTimestamp(now()), 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 1, 'Lucy in the Sky with Diamonds', 208);
INSERT INTO tracks_by_user (id, month, timestamp, album_title, album_year, number, title, length) VALUES (uuid(), '2024-01-01', toTimestamp(now()), 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 2, 'With a Little Help from My Friends', 163);
INSERT INTO tracks_by_user (id, month, timestamp, album_title, album_year, number, title, length) VALUES (uuid(), '2024-01-01', toTimestamp(now()), 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 3, 'Sgt. Pepper''s Lonely Hearts Club Band', 122);
INSERT INTO tracks_by_user (id, month, timestamp, album_title, album_year, number, title, length) VALUES (uuid(), '2024-01-01', toTimestamp(now()), 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 4, 'Getting Better', 174);
INSERT INTO tracks_by_user (id, month, timestamp, album_title, album_year, number, title, length) VALUES (uuid(), '2024-01-01', toTimestamp(now()), 'Sgt. Pepper''s Lonely Hearts Club Band', 1967, 5, 'Fixing a Hole', 139);

"""

In [ ]:
# This parses the text above into executable strings by the driver
for line in insert_fake_data_cql.split("\n"):
    sc_loc = line.find(";")
    if sc_loc > -1:
        execute_statement(line[:sc_loc])

## (Optional) Give the LLM Additional Context with the Built-in 'Comments' Column

LLM response quality greatly depends on the context they've been given - the more concise descriptions they have access to, the better. We can choose to augment the DB schema we pass to the model by utilizing the built-in `comment` property of CQL tables.

NOTE: You can also include these comments at table creation by using the `WITH <table property 1> AND <table property 2> ... AND comment = '<comment>'` syntax

In [ ]:
add_comments_cql = f"""
ALTER TABLE albums_by_genre WITH comment = 'Albums partitioned by musical genre';
ALTER TABLE albums_by_performer WITH comment = 'Albums partitioned by name of performer/artist';
ALTER TABLE albums_by_title WITH comment = 'Albums partitioned by album title';
ALTER TABLE performers WITH comment = 'Performers/artists partitioned by performer name';
ALTER TABLE tracks_by_album WITH comment = 'Tracks/songs partitioned by album title';
ALTER TABLE tracks_by_title WITH comment = 'Tracks/songs partitioned by song title';
ALTER TABLE tracks_by_user WITH comment = 'Tracks/songs users listened to partitioned by user ID and time of listen';
ALTER TABLE users WITH comment = 'Users partitioned by user ID';
"""

In [ ]:
# This parses the text above into executable strings by the driver
for line in add_comments_cql.split("\n"):
    sc_loc = line.find(";")
    if sc_loc > -1:
        execute_statement(line[:sc_loc])

## Run Queries from User Questions

#### Generating & Executing CQL

Now, we can ask ChatGPT to provide us with some queries that answer our questions! The prompt template we use is taken from [SQL-PaLM](https://arxiv.org/abs/2306.00739), and adapted to fit the CQL use case. In order to use it though, we need to retrieve the schema from our DB.

In [ ]:
TEXT2CQL_PROMPT = """Convert the question to CQL (Cassandra Query Language) that can retrieve an appropriate answer, or answer saying that the data model does not support answering such a question in a performant way:

[Schema : values (type)]
{schema}

[Partition Keys]
{partition_keys}

[Clustering Keys]
{clustering_keys}

[Q]
{question}

[CQL]
"""


def generate_schema_partition_clustering_keys(keyspace: str = ASTRA_KEYSPACE) -> (str, str):
    """Generates a TEXT2CQL_PROMPT compatible schema for a keyspace"""
    # Get all table names in our keyspace
    table_names = execute_statement(
        f"SELECT table_name, comment FROM system_schema.tables WHERE keyspace_name = '{keyspace}'"
    )
    tn_str = ", ".join(["'" + tn.table_name + "'" for tn in table_names])

    # Now get all the column names corresponding to those tables
    columns = execute_statement(
        f"SELECT * FROM system_schema.columns WHERE table_name IN ({tn_str}) AND keyspace_name = '{keyspace}' ALLOW FILTERING"
    )

    # Now, we construct our prompt template formatted schema, partition_keys, and clustering keys
    # from the table and column objects returned from the DB
    schema = " | ".join([
        f"{table.table_name} '{table.comment}' : " + " , ".join([
            f"{col.column_name} ({col.type})"
            for col in columns
            if col.table_name == table.table_name
        ])
        for table in table_names
    ])
    partition_keys = " | ".join([
        f"{table.table_name} : " + " , ".join([
            col.column_name for col in columns
            if col.table_name == table.table_name
            and col.kind == "partition_key"
        ])
        for table in table_names
    ])
    clustering_keys = " | ".join([
        f"{table.table_name} : " + " , ".join([
            f"{col.column_name} ({col.clustering_order})" for col in columns
            if col.table_name == table.table_name
            and col.kind == "clustering"
        ])
        for table in table_names
    ])
    return schema, partition_keys, clustering_keys


def execute_query_from_question(question: str, debug_cql: bool = True, debug_prompt: bool = False, return_cql: bool = False):
    """Generates and executes CQL from a user question based on LLM output"""
    # Get all of the variables necessary to fill out the prompt
    schema, partition_keys, clustering_keys = generate_schema_partition_clustering_keys()
    prompt = TEXT2CQL_PROMPT.format(
        schema=schema,
        partition_keys=partition_keys,
        clustering_keys=clustering_keys,
        question=question,
    )

    if debug_prompt:
        print(f"Prompting model with:\n{prompt}")

    # Get generated CQL from the LLM (in this case gpt-3.5-turbo)
    completion = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        model="gpt-3.5-turbo",
    ).choices[0].message.content

    if debug_cql:
        print(f"Question: {question}\nGenerated Query: {completion}\n")

    # Need to trim trailing ';' if present to work with cassandra-driver
    if completion.find(";") > -1:
        completion = completion[:completion.find(";")]

    results = execute_statement(completion)

    if return_cql:
        return (results, completion)
    else:
        return results

In [ ]:
# Show full prompting trace
execute_query_from_question("What songs are on A Night at the Opera?", debug_prompt=True)

Prompting model with:
Convert the question to CQL (Cassandra Query Language) that can retrieve an appropriate answer, or answer saying that the data model does not support answering such a question in a performant way:

[Schema : values (type)]
albums_by_genre 'Albums partitioned by musical genre' : genre (text) , performer (text) , title (text) , year (int) | albums_by_performer 'Albums partitioned by name of performer/artist' : genre (text) , performer (text) , title (text) , year (int) | albums_by_title 'Albums partitioned by album title' : genre (text) , performer (text) , title (text) , year (int) | performers 'Performers/artists partitioned by performer name' : born (int) , country (text) , died (int) , founded (int) , name (text) , type (text) | tracks_by_album 'Tracks/songs partitioned by album title' : album_title (text) , album_year (int) , genre (text) , length (int) , number (int) , title (text) | tracks_by_title 'Tracks/songs partitioned by song title' : album_title (text)

[Row(title='Bohemian Rhapsody'),
 Row(title='Love of My Life'),
 Row(title='Youre My Best Friend')]

In [ ]:
execute_query_from_question("What are some of the most recent Pop albums in the last decade?")

Question: What are some of the most recent Pop albums in the last decade?
Generated Query: SELECT title, year FROM albums_by_genre WHERE genre = 'Pop' AND year >= 2010 ALLOW FILTERING;



[Row(title='25', year=2015), Row(title='1989', year=2014)]

In [ ]:
execute_query_from_question("How many albums has Taylor Swift made?")

Question: How many albums has Taylor Swift made?
Generated Query: SELECT COUNT(*) FROM albums_by_performer WHERE performer = 'Taylor Swift';



[Row(count=1)]

Pretty cool that it can find the data to answer our questions! Let's see if we can take this one step further, and actually generate coherent responses using this data:

#### End to End Question Answering

Now, let's wrap up by showing how we can make a subsequent LLM call to answer the user's question with natural language. This completes a full "RAG" style pipeline!

In [ ]:
ANSWER_PROMPT = """Query:
```
{cql}
```

Output:
```
{results_repr}
```
===

Given the above results from querying the DB, answer the following user question:

{question}
"""


def answer_question(question: str, debug_cql: bool = False, debug_prompt: bool = False) -> str:
    """Conducts a full RAG pipeline where the LLM retrieves relevant information
    and references it to answer the question in natural language.
    """
    # Get necessary fields to fill out prompt
    query_results, cql = execute_query_from_question(
        question=question,
        debug_cql=debug_cql,
        debug_prompt=debug_prompt,
        return_cql=True,
    )
    prompt = ANSWER_PROMPT.format(
        question=question,
        results_repr=str(query_results),
        cql=cql,
    )

    if debug_prompt:
        print(f"Prompting model with:\n{prompt}")

    # Return the generated answer from the LLM
    return client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        model="gpt-3.5-turbo",
    ).choices[0].message.content


In [ ]:
# Show full prompting trace
print(
    answer_question("What songs are on A Night at the Opera?", debug_prompt=True)
)

Prompting model with:
Convert the question to CQL (Cassandra Query Language) that can retrieve an appropriate answer, or answer saying that the data model does not support answering such a question in a performant way:

[Schema : values (type)]
albums_by_genre 'Albums partitioned by musical genre' : genre (text) , performer (text) , title (text) , year (int) | albums_by_performer 'Albums partitioned by name of performer/artist' : genre (text) , performer (text) , title (text) , year (int) | albums_by_title 'Albums partitioned by album title' : genre (text) , performer (text) , title (text) , year (int) | performers 'Performers/artists partitioned by performer name' : born (int) , country (text) , died (int) , founded (int) , name (text) , type (text) | tracks_by_album 'Tracks/songs partitioned by album title' : album_title (text) , album_year (int) , genre (text) , length (int) , number (int) , title (text) | tracks_by_title 'Tracks/songs partitioned by song title' : album_title (text)

In [ ]:
print(
    answer_question("What are some of the most recent Pop albums in the last decade?")
)

Some of the most recent Pop albums in the last decade are "25" by Adele (released in 2015) and "1989" by Taylor Swift (released in 2014).


In [ ]:
print(
    answer_question("How many albums has Taylor Swift made?")
)

Taylor Swift has made 1 album.


Awesome! Our model is answering questions based on just the data in our dummy DB, and is able to construct queries for retrieving that data in a fully automated way.